http://www.jessicayung.com/using-generators-in-python-to-train-machine-learning-models/


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
print (os.getcwd())

/content


In [0]:
# Part 1 - Building the CNN

# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense

In [0]:
# Initialising the CNN
classifier = Sequential()

# Step 1 - Convolution
classifier.add(Conv2D(32, (3, 3), input_shape = (64, 64, 3), activation = 'relu'))

# Step 2 - Pooling
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Adding a second convolutional layer
classifier.add(Conv2D(32, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Step 3 - Flattening
classifier.add(Flatten())

# Step 4 - Full connection
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dense(units = 3, activation = 'softmax'))

# Compiling the CNN
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])


In [0]:
classifier.summary()

In [0]:

# Part 2 - Fitting the CNN to the images

from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('/content/drive/My Drive/dataset/training_set',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory('/content/drive/My Drive/dataset/test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'categorical')



Found 445 images belonging to 3 classes.
Found 238 images belonging to 3 classes.


In [0]:
type(training_set)
type(train_datagen)

keras.preprocessing.image.ImageDataGenerator

In [0]:
classifier.fit_generator(training_set,
                         steps_per_epoch = 100,
                         epochs = 5,#25
                         validation_data = test_set,
                         validation_steps = 200)

Instructions for updating:
Use tf.cast instead.
Epoch 1/5
  6/100 [>.............................] - ETA: 35:21 - loss: 0.7769 - acc: 0.6458

/usr/local/lib/python3.6/dist-packages/PIL/Image.py:914: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


100/100 [==============================] - 543s 5s/step - loss: 0.2912 - acc: 0.8719 - val_loss: 0.1487 - val_acc: 0.9440
Epoch 2/5
100/100 [==============================] - 262s 3s/step - loss: 0.0862 - acc: 0.9708 - val_loss: 0.1651 - val_acc: 0.9426
Epoch 3/5
100/100 [==============================] - 261s 3s/step - loss: 0.0495 - acc: 0.9822 - val_loss: 0.1620 - val_acc: 0.9510
Epoch 4/5
100/100 [==============================] - 262s 3s/step - loss: 0.0265 - acc: 0.9896 - val_loss: 0.1412 - val_acc: 0.9608
Epoch 5/5
100/100 [==============================] - 262s 3s/step - loss: 0.0165 - acc: 0.9952 - val_loss: 0.2293 - val_acc: 0.9440


## Using the pretrained model: VGG16

In [0]:
from keras.applications import VGG16

In [0]:
conv_base = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

58892288/58889256 [==============================] - 4s 0us/step


In [0]:
conv_base.summary()

In [0]:
from keras import models
from keras import layers
import os
model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(3, activation='softmax'))

In [0]:
conv_base.trainable = False #do not train the conv-base

In [0]:
base_dir = '/content/drive/My Drive/dataset/'
train_dir = os.path.join(base_dir, 'training_set')

test_dir = os.path.join(base_dir, 'test_set')

In [0]:
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers

train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=40, width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255) #rescale factor is feature scalling, convert values in the range of 0-1 before applying any processing

In [0]:
train_generator = train_datagen.flow_from_directory(train_dir, 
                                                    target_size=(150, 150),
                                                    batch_size=20,
                                                    class_mode='categorical')

Found 445 images belonging to 3 classes.


In [0]:
test_generator = test_datagen.flow_from_directory(test_dir,
                                                  target_size=(150, 150),
                                                  batch_size=20,
                                                  class_mode='categorical')

Found 238 images belonging to 3 classes.


In [0]:
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=2e-5),
              metrics=['acc'])

In [0]:
history = model.fit_generator(train_generator, steps_per_epoch=100,
                              epochs=30,
                              validation_data=test_generator,
                              validation_steps=50)

Epoch 1/30
  4/100 [>.............................] - ETA: 2:26 - loss: 1.0805 - acc: 0.4000

/usr/local/lib/python3.6/dist-packages/PIL/Image.py:914: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


100/100 [==============================] - 101s 1s/step - loss: 0.2541 - acc: 0.8885 - val_loss: 0.0936 - val_acc: 0.9667
Epoch 2/30
100/100 [==============================] - 91s 912ms/step - loss: 0.0694 - acc: 0.9800 - val_loss: 0.0311 - val_acc: 0.9879
Epoch 3/30
100/100 [==============================] - 92s 919ms/step - loss: 0.0219 - acc: 0.9940 - val_loss: 0.0427 - val_acc: 0.9909
Epoch 4/30
100/100 [==============================] - 93s 928ms/step - loss: 0.0210 - acc: 0.9915 - val_loss: 0.0500 - val_acc: 0.9960
Epoch 5/30
100/100 [==============================] - 91s 911ms/step - loss: 0.0186 - acc: 0.9965 - val_loss: 0.0576 - val_acc: 0.9950
Epoch 6/30
100/100 [==============================] - 92s 921ms/step - loss: 0.0123 - acc: 0.9970 - val_loss: 0.0366 - val_acc: 0.9960
Epoch 7/30
100/100 [==============================] - 92s 921ms/step - loss: 0.0069 - acc: 0.9965 - val_loss: 0.0259 - val_acc: 0.9960
Epoch 8/30
100/100 [==============================] - 92s 915ms/step

### Using a pre trained model: Fine Tuning the pre trained model

Another widely used technique for model reuse, complementary to feature
extraction, is fine-tuning (see figure 5.19). Fine-tuning consists of unfreezing a few of
the top layers of a frozen model base used for feature extraction, and jointly training
both the newly added part of the model (in this case, the fully connected classifier)
and these top layers. This is called fine-tuning because it slightly adjusts the more
abstract representations of the model being reused, in order to make them more relevant
for the problem at hand.

Thus the
steps for fine-tuning a network are as follow:
1 Add your custom network on top of an already-trained base network.
2 Freeze the base network.
3 Train the part you added.
4 Unfreeze some layers in the base network.
5 Jointly train both these layers and the part you added.
You already completed the first three steps when doing feature extraction. Let’s proceed
with step 4: you’ll unfreeze your conv_base and then freeze individual layers
inside it.

You’ll fine-tune the last three convolutional layers, which means all layers up to
block4_pool should be frozen, and the layers block5_conv1, block5_conv2, and
block5_conv3 should be trainable.

In [0]:
conv_base.trainable = True
set_trainable = False
for layer in conv_base.layers:
  if layer.name == 'block5_conv1':
    set_trainable = True
    
  if set_trainable:
    layer.trainable = True
  else:
    layer.trainable = False

NameError: ignored

In [0]:
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=2e-5),
              metrics=['acc'])

history = model.fit_generator(
    train_generator,
    steps_per_epoch=100,
    epochs=30,
    validation_data=test_generator,
    validation_steps=50)

Here’s what you should take away from the exercises in the past two sections:

Convnets are the best type of machine-learning models for computer-vision
tasks. It’s possible to train one from scratch even on a very small dataset, with
decent results.

On a small dataset, overfitting will be the main issue. Data augmentation is a
powerful way to fight overfitting when you’re working with image data.

It’s easy to reuse an existing convnet on a new dataset via feature extraction.
This is a valuable technique for working with small image datasets.

As a complement to feature extraction, you can use fine-tuning, which adapts to
a new problem some of the representations previously learned by an existing
model. This pushes performance a bit further.

In [0]:
divmod(23,5)

(4, 3)